### Dijkstra's Algorithm

- Dijkstra's algorithm finds the shortest path between from some source vertex to all destination vertices in a graph, given a set of edge weights
    - Dijkstra's works for directed and undirected graphs
    - BUT does not work when the edges contain negative values

### How does this work?

- Let's suppose we start at a given vertex $V_i$

- We create an adjacency map from every vertex $V_i$ to every other vertex $V_{j \neq i}$, including their weight. That is
    - $\{ V_i: [(V_j, 1), (V_k, 2)] \}$
    - This will tell us, for every vertex $V_i$, which are its immediate neighbours, and the edge weight between them

- We create array `shortest_distance_from_vi` of size $|V|$, with each index representing a vertex in the graph
    - Each index `j` in this array represents the distance of the vertex $V_j$ from $V_i$
    - Since not every other vertex is connected to $V_i$, we initialise all values as $\inf$
    - We will iteratively traverse all nodes in the graph to update these distances 

- Starting from $V_i$, we assign the value at `shortest_distance_from_vi[i]` to be 0. Intuitively, this just means that the distance of vertex `i` from itself is 0

- Next, we update the distances for all neighbours of $V_i$ in `shortest_distance_from_vi`. 
    - This can be done by taking the weight at $V_i$ and adding it to the edge value of the neighbour
        - That is, suppose it costs us 2 to reach $V_i$, and the edge between $V_i$ and $V_j$ is 2, then the total cost to reach $V_j$ through $V_i$ must be 2+2 = 4
        - As we iterate through the vertices, it is possible that we encounter $V_i + E_{ij} > V_j$
        - In such a case, don't update `shortest_distance_from_vi`
    
    - This process is called **relaxation**

- We pick a neighbour with the smallest distance from $V_i$, and run the same algorithm on it
    - Before moving on to the smallest neighbour, mark $V_i$ as visited. We will never visit $V_i$ again

### Example Walkthrough

- Let's use the graph below

In [40]:
import matplotlib.pyplot as plt
import networkx as nx

G = nx.DiGraph()

G.add_edge("1", "2", weight=50)
G.add_edge("1", "3", weight=45)
G.add_edge("1", "4", weight=10)
G.add_edge("2", "4", weight=15)
G.add_edge("2", "3", weight=10)
G.add_edge("3", "5", weight=30)
G.add_edge("4", "5", weight=15)
G.add_edge("5", "2", weight=20)
G.add_edge("5", "3", weight=35)
G.add_edge("5", "1", weight=1)
G.add_edge("6", "5", weight=3)

# plt.figure(figsize=(12,7))
# pos = nx.spring_layout(G, k=10, iterations=20, seed=7)  # positions for all nodes - seed for reproducibility
# nx.draw_networkx_nodes(G, pos, cmap=plt.get_cmap('jet'), node_size = 500)
# nx.draw_networkx_labels(G, pos, font_color='white')
# nx.draw_networkx_edges(G, pos, arrows=True, arrowsize=15)
# nx.draw_networkx_edge_labels(G, pos, font_size='12')
# plt.show();

- Let's assume we start from node 1

    - Add node 1 to `visited` set. No matter what, we will never visit node 1 again
        - Why?
        - Recall that at every iteration, we take the shortest possible next step
        - And in Dijkstra's, the assumption is that every edge is positive
        - So for any node `i` that we reach, it will not be possible to find another path that will get us to this node with less cost
        - Because any separate path we take must contain a larger edge than the current path. And since edge values are non negative, it is not possible to get a smaller weight from making more hops 

    - So we see that there 1 has 3 neighbours, 2 (with weight 50), 3 (with weight 45), and 4 (with weight 10)

    - `shortest_distance_from_vi = [0,50,45,10,inf,inf]`
    - `visited = {1}`

- So next one with visit is 4
    - `visited = {1,4}`
    - 4 has only 1 neighbour 5 (with weight 15)
    - 5 is not in visited, so perform **relaxation**
    - `shortest_distance_from_vi = [0,50,45,10,10+15=25,inf] = [0,50,45,10,25,inf]`

- Visit 5
    - `visited = {1,4,5}`
    - 5 has 3 neighbours; 1 (with edge 1), 2 (with edge 20), and 3 (with edge 35)
    - 1 already exists in `visited`, so skip
    - The total distance of path to 2 that goes through 5 is 25 + 20 = 45. This is smaller than 50, so update
    - `shortest_distance_from_vi = [0,45,45,10,25,inf]`
    - The total distance of path to 3 that goes through 5 is 25 + 35 = 60. This is larger than 45, don't update
    - 1 is the next smallest next edge, but 1 is already visited. Ignore
    - 2 and 3 have the same weight, just choose one

- Visit 2
    - `visited = {1,4,5,2}`
    - 2 has 2 neighbours; 3 (with edge 10), and 4 (with edge 15)
    - The total distance of path to 3 that goes through 2 is 45 + 10 = 55. This is longer than the existing value of 45, so don't update
    - 4 is already in `visisted`, skip
    - `shortest_distance_from_vi = [0,45,45,10,25,inf]`

- Visit 3
    - `visited = {1,4,5,2,3}`
    - 3 has 1 neighbour; 5 (with edge 30)
    - 5 is already visisted, skip
    - There are no neighbours that have not been visited, end

### Why does Dijkstra's fail with negative edge?

- Let's see a case now where Dijkstra's will fail with negative edge

In [42]:
import networkx as nx

G = nx.DiGraph()
G.add_edge("1", "2", weight=3)
G.add_edge("1", "4", weight=5)
G.add_edge("3", "2", weight=-10)
G.add_edge("4", "3", weight=2)

# plt.figure(figsize=(12,7))
# pos = nx.spring_layout(G, k=10, iterations=20, seed=7)  # positions for all nodes - seed for reproducibility
# nx.draw_networkx_nodes(G, pos, cmap=plt.get_cmap('jet'), node_size = 500)
# nx.draw_networkx_labels(G, pos, font_color='white')
# nx.draw_networkx_edges(G, pos, arrows=True, arrowsize=15)
# nx.draw_networkx_edge_labels(G, pos, font_size='12')
# plt.show();

- Again, start from node 1
    - `visited = {1}`
    - `shortest_distance_from_vi = [0,inf,inf,inf]`
    - 2 neighbours; 2 (with edge 3), 4 (with edge 5)
    - Update both indices in shortest_distance array
    - `shortest_distance_from_vi = [0,3,inf,5]`

- Visit 2
    - `visited = {1,2}`
    - `shortest_distance_from_vi = [0,3,inf,5]`
    - No neighbours, so no updates

- Visit 4
    - `visited = {1,2,4}`
    - 1 neighbour; 3 (with edge 2)
    - Path to 3 through 4 is 5 + 2 = 7
    - `shortest_distance_from_vi = [0,3,7,5]`

- Visit 3
    - `visited = {1,2,4,3}`
    - Neighbour 2 has been visisted, skip
    - `shortest_distance_from_vi = [0,3,7,5]`

- But wait, let's zoom into the **Visit 3** chunk
    - Suppose we didn't skip this.
    - We would have found that the shortest path to neighbour 2 through 3 is 7 - 10 = -3
    - And this value is smaller than the one at the 2 index!!!
    - So ideally, the shortest distance array should be:
        - `shortest_distance_from_vi = [0,-3,7,5]`

- So the assumption that more hops = more distance that Dijkstra's algorithm makes will fail when negative weights are introduced

### Code Implementation

- Let's implement Dijkstra's in the following way

- We will provide an array of edges and edge weights i.e. `(from, to, weight)`

- Given this information, use Dijkstra's to find the shortest path from some arbitrary vertex $i$ to all other vertices $j$

- In this implementation, we assume that all edges are bidirectional

In [79]:
## (from, to, edge_weight)
inputs = [
    (0,1,4), (0,7,8), (1,2,8), (1,7,11),
    (2,3,7), (2,8,2), (2,5,4), (3,4,9),
    (3,5,14), (4,5,10), (5,6,2), (6,7,1),
    (6,8,6), (7,8,7)
]

def construct_adjacency_map(inputs: list[tuple[int,int,int]]) -> dict[int, tuple[int, int]]:
    '''
    Function should produce an adjacency map of {from: (weight, to)}. This format wll help align with Dijkstra's minheap structure later
    '''
    adj_map = {}
    for input in inputs:
        from_node, to_node, edge_weight = input
        if from_node not in adj_map:
            adj_map[from_node] = [(edge_weight, to_node)]
        else:
            adj_map[from_node].append((edge_weight,to_node))

        if to_node not in adj_map:
            adj_map[to_node] = [(edge_weight, from_node)]
        else:
            adj_map[to_node].append((edge_weight,from_node))

    return adj_map

adj_map = construct_adjacency_map(inputs)
adj_map

{0: [(4, 1), (8, 7)],
 1: [(4, 0), (8, 2), (11, 7)],
 7: [(8, 0), (11, 1), (1, 6), (7, 8)],
 2: [(8, 1), (7, 3), (2, 8), (4, 5)],
 3: [(7, 2), (9, 4), (14, 5)],
 8: [(2, 2), (6, 6), (7, 7)],
 5: [(4, 2), (14, 3), (10, 4), (2, 6)],
 4: [(9, 3), (10, 5)],
 6: [(2, 5), (1, 7), (6, 8)]}

In [78]:
import math
import heapq

def dijkstras(start_node: int, adj_map: dict) -> list[int]:
    distance_from_start_node = [math.inf for _ in adj_map]
    distance_from_start_node[start_node] = 0
    visited = set()

    ## Queue tuples are (edge_weight, next_node_to_visit)
    queue = [(0, start_node)] 
    
    while queue:
        curr_node_distance, curr_node = heapq.heappop(queue)
        visited.add(curr_node)
        
        for neighbour_edge_val, neighbour_node in adj_map.get(curr_node, []):
            if neighbour_node in visited:
                continue
                        
            if distance_from_start_node[neighbour_node] > (curr_node_distance + neighbour_edge_val):
                distance_from_start_node[neighbour_node] = curr_node_distance + neighbour_edge_val
                heapq.heappush(queue, (distance_from_start_node[neighbour_node], neighbour_node))
    
    return distance_from_start_node

dijkstras(0, adj_map)

[0, 4, 12, 19, 21, 11, 9, 8, 14]

### Time complexity of Dijkstra's Algorithm

- Let's think through the overall algorithm
    1. We iterate through all vertices in the graph to construct the `distance_from_start_node` array
        - This is done in $O(|V|)$ time
    2. We create a heap, and push every node onto the heap
        - This is done in $O(log(|V|))$ time
    3. For each popped vertex from the queue, we iterate through its neighbours. In the worst case, it is fully connected, so iterating through all neighbours takes $O(||)$ time
